In [2]:
from xgboost import XGBRegressor
from Predictions import Regressor
import re
import datetime
import math
import pandas as pd

params = {
    'tree_method':"auto", 
    'objective':'reg:squarederror', 
    'learning_rate':1, 
    'verbosity':0, 
    'early_stopping_rounds':5
}
# initializing xgboost classifier
classifier = Regressor(params)
dataFile = open("2017-weather.txt", 'r')
for line in dataFile:
    x = re.findall("-\d+\.\d+|\d+\.\d+|\d+|[A-Z]", line)
    year = re.match("[0-9]{4}", x[1])
    year = int(year.group(0))
    month = re.match("[0-9]{4}([0-9]{2})", x[1])
    month = int(month.group(1))
    day = re.match("[0-9]{4}[0-9]{2}([0-9]{2})", x[1])
    day = int(day.group(1))
    monthDay = re.match("[0-9]{4}([0-9]{4})", x[1])
    date = datetime.datetime.strptime(monthDay.group(1), "%m%d")
    dayOfYear = int(date.timetuple().tm_yday) - 1
    sin = math.sin(dayOfYear)
    cos = math.cos(dayOfYear)
    hour = int(x[2])
    latitude = float(x[7])
    longitude = float(x[6])
    temperature = float(x[8])
    # input the data
    classifier.input_train([year, month, day, hour, sin, cos, latitude, longitude, temperature], float(x[9]))
dataFile.close()

# process the testing data
dataFile = open("2018-weather.txt", 'r')
monthlist = []
actual = []
for line in dataFile:
    x = re.findall("-\d+\.\d+|\d+\.\d+|\d+|[A-Z]", line)
    year = re.match("[0-9]{4}", x[1])
    year = int(year.group(0))
    month = re.match("[0-9]{4}([0-9]{2})", x[1])
    month = int(month.group(1))
    monthlist.append(month)
    day = re.match("[0-9]{4}[0-9]{2}([0-9]{2})", x[1])
    monthDay = re.match("[0-9]{4}([0-9]{4})", x[1])
    date = datetime.datetime.strptime(monthDay.group(1), "%m%d")
    dayOfYear = int(date.timetuple().tm_yday) - 1
    sin = math.sin(dayOfYear)
    cos = math.cos(dayOfYear)
    day = int(day.group(1))
    hour = int(x[2])
    latitude = float(x[7])
    longitude = float(x[6])
    temperature = float(x[8])
    # input the data
    classifier.input_test([year, month, day, hour, sin, cos, latitude, longitude, temperature])
    classifier.label(float(x[9]))
    actual.append(float(x[9]))

dataFile.close()

print("Training, this will take a few minutes")
print(classifier.train_eval("mae"))

print("Predicting, this will take a minute")
predictions = classifier.predict()
print("Predictions: ", predictions)

Training, this will take a few minutes
[0]	validation_0-mae:0.33542	validation_1-mae:1.54849
[1]	validation_0-mae:0.07414	validation_1-mae:1.41924
[2]	validation_0-mae:0.03582	validation_1-mae:1.42721
[3]	validation_0-mae:0.02397	validation_1-mae:1.44064
[4]	validation_0-mae:0.01902	validation_1-mae:1.44883
[5]	validation_0-mae:0.01716	validation_1-mae:1.45374
[6]	validation_0-mae:0.01605	validation_1-mae:1.45613
[7]	validation_0-mae:0.01557	validation_1-mae:1.45752
[8]	validation_0-mae:0.01587	validation_1-mae:1.45866
[9]	validation_0-mae:0.01573	validation_1-mae:1.45903
[10]	validation_0-mae:0.01541	validation_1-mae:1.45904
[11]	validation_0-mae:0.01537	validation_1-mae:1.45922
[12]	validation_0-mae:0.01501	validation_1-mae:1.45920
[13]	validation_0-mae:0.01514	validation_1-mae:1.45923
[14]	validation_0-mae:0.01474	validation_1-mae:1.45915
[15]	validation_0-mae:0.01462	validation_1-mae:1.45914
[16]	validation_0-mae:0.01477	validation_1-mae:1.45934
[17]	validation_0-mae:0.01476	valida

In [2]:
data = {"month": monthlist,
       "actual": actual,
       "predictions": predictions}

In [3]:
df = pd.DataFrame(data, columns = ["month", "actual", "predictions"])

In [4]:
df = df.drop_duplicates(subset=['month'], keep='first')

In [5]:
df

,month,actual,predictions
0,1,0.0,-0.008937
8927,2,0.0,0.012435
16991,3,0.0,0.022566
25919,4,0.0,0.012751
34559,5,0.0,0.032769
43487,6,0.0,0.028339
52127,7,0.0,0.012478
61055,8,0.0,0.027112
69983,9,0.0,0.028214
78623,10,0.0,-0.046501


In [6]:
import altair as alt
line = alt.Chart(df).mark_line().encode(
    x='month',
    y='predictions'
)
scatter = alt.Chart(df).mark_circle().encode(
    x='month',
    y='actual'
)
graph = line + scatter
graph
#line

alt.LayerChart(...)

In [7]:
scatter = alt.Chart(df).mark_circle().encode(
    x='predictions',
    y='actual'
)
scatter

alt.Chart(...)

In [8]:
scatter = alt.Chart(df).mark_circle().encode(
    y='predictions',
    x='actual'
)
scatter

alt.Chart(...)

In [9]:
line = alt.Chart(df).mark_line().encode(
    x='actual',
    y='predictions'
)
scatter = alt.Chart(df).mark_circle(size=60).encode(
    x='actual',
    y='predictions'
)
graph = line + scatter
graph

alt.LayerChart(...)

In [134]:
from Predictions import Regressor
import re
import datetime
import math
import pandas as pd

params = {
    'max_depth':8,
    'min_child_weight': 0.001,
    'eta':.5,
    'subsample': 1,
    'colsample_bytree': 0.99,
    'objective':'reg:squarederror',
    'verbosity':0,
    'early_stopping_rounds':1,
    'learning_rate':1.5,
    'tree_method':"gpu_hist"
}

# initializing xgboost classifier
classifier = Regressor(params)

dataFile = open("2017-weather.txt", 'r')
for line in dataFile:
    x = re.findall("-\d+\.\d+|\d+\.\d+|\d+|[A-Z]", line)
    year = re.match("[0-9]{4}", x[1])
    year = int(year.group(0))
    month = re.match("[0-9]{4}([0-9]{2})", x[1])
    month = int(month.group(1))
    day = re.match("[0-9]{4}[0-9]{2}([0-9]{2})", x[1])
    day = int(day.group(1))
    monthDay = re.match("[0-9]{4}([0-9]{4})", x[1])
    date = datetime.datetime.strptime(monthDay.group(1), "%m%d")
    dayOfYear = int(date.timetuple().tm_yday) - 1
    sin = math.sin(dayOfYear)
    cos = math.cos(dayOfYear)
    hour = int(x[2])
    latitude = float(x[7])
    longitude = float(x[6])
    temperature = float(x[8])
    # input the data
    classifier.input_train([year, month, day, hour, sin, cos, latitude, longitude], temperature)
dataFile.close()

# process the testing data
dataFile = open("2018-weather.txt", 'r')
monthlist = []
actual = []
for line in dataFile:
    x = re.findall("-\d+\.\d+|\d+\.\d+|\d+|[A-Z]", line)
    year = re.match("[0-9]{4}", x[1])
    year = int(year.group(0))
    month = re.match("[0-9]{4}([0-9]{2})", x[1])
    month = int(month.group(1))
    monthlist.append(month)
    day = re.match("[0-9]{4}[0-9]{2}([0-9]{2})", x[1])
    monthDay = re.match("[0-9]{4}([0-9]{4})", x[1])
    date = datetime.datetime.strptime(monthDay.group(1), "%m%d")
    dayOfYear = int(date.timetuple().tm_yday) - 1
    sin = math.sin(dayOfYear)
    cos = math.cos(dayOfYear)
    day = int(day.group(1))
    hour = int(x[2])
    latitude = float(x[7])
    longitude = float(x[6])
    temperature = float(x[8])
    # input the data
    classifier.input_test([year, month, day, hour, sin, cos, latitude, longitude])
    classifier.label(temperature)
    actual.append(temperature)

dataFile.close()

print("Training, this will take a few minutes")
classifier.train_eval("mae")

print("Predicting, this will take a minute")
predictions = classifier.predict()
print("Predictions: ", predictions)

Training, this will take a few minutes
[0]	validation_0-mae:11.51013	validation_1-mae:11.59566
[1]	validation_0-mae:7.98339	validation_1-mae:8.73264
[2]	validation_0-mae:6.35930	validation_1-mae:7.89602
[3]	validation_0-mae:5.60788	validation_1-mae:7.33405
[4]	validation_0-mae:5.28041	validation_1-mae:7.17659
[5]	validation_0-mae:4.99861	validation_1-mae:7.56636
[6]	validation_0-mae:4.90217	validation_1-mae:7.61037
[7]	validation_0-mae:4.94833	validation_1-mae:7.63872
[8]	validation_0-mae:4.48179	validation_1-mae:7.41791
[9]	validation_0-mae:4.18125	validation_1-mae:7.33466
[10]	validation_0-mae:4.10494	validation_1-mae:7.31304
[11]	validation_0-mae:3.89105	validation_1-mae:7.25256
[12]	validation_0-mae:3.72047	validation_1-mae:7.12439
[13]	validation_0-mae:3.56387	validation_1-mae:7.10574
[14]	validation_0-mae:3.48568	validation_1-mae:7.06275
[15]	validation_0-mae:3.32086	validation_1-mae:7.03991
[16]	validation_0-mae:3.22873	validation_1-mae:7.02750
[17]	validation_0-mae:3.14706	vali

In [166]:
classifier.check_error()

17.964505243025936

In [144]:

data = {"month": monthlist,
       "actual": actual,
       "prediction": predictions}
df = pd.DataFrame(data, columns = ["month", "actual", "prediction"])


print(len(df))

df3 = df.sample(n=5000, random_state=1)
len(df3)

105120


5000

In [138]:


# monthlist2 = monthlist[3000:5000]
# actual2 = actual[3000:5000]
# predictions2 = predictions[3000:5000]

data = {"month": monthlist2,
       "actual": actual2,
       "prediction": predictions2}
df = pd.DataFrame(data, columns = ["month", "actual", "prediction"])
# df = df.drop_duplicates(subset=['month'], keep='last')
print(len(df))
# df = df.groupby('month').mean()
# df['month'] = df.index
d = {'actual': [-30, 30], 'prediction': [-30, 30]}
df2 = pd.DataFrame(data=d)


2000


In [136]:
# import seaborn as sns
# data = {"month": monthlist,
#        "actual": actual,
#        "predictions": predictions}
# df3 = pd.DataFrame(data, columns = ["month", "actual", "predictions"])
# sns.lmplot(x="actual", y="predictions", data=df3)

In [155]:
# line = alt.Chart(df2).mark_line().encode(
#     x=alt.X('actual',
#            scale=alt.Scale(domain=(-25, 30))),
#     y=alt.Y('predictions',
#            scale=alt.Scale(domain=(-25, 30)))
# )
# scatter = alt.Chart(df).mark_circle(size=60).encode(
#     x=alt.X('actual',
#            scale=alt.Scale(domain=(-25, 30))),
#     y=alt.Y('predictions',
#            scale=alt.Scale(domain=(-25, 30)))
# )
# graph = line + scatter
# graph
line = alt.Chart(df2).mark_line().encode(
    x=alt.X('actual:Q'),
    y=alt.Y('prediction:Q')
)
scatter = alt.Chart(df).mark_circle().encode(
    x=alt.X('actual:Q'),
    y=alt.Y('prediction:Q')
)
graph = line + scatter
graph

alt.LayerChart(...)

In [163]:
data = {"month": monthlist,
       "actual": actual,
       "prediction": predictions}
df = pd.DataFrame(data, columns = ["month", "actual", "prediction"])

df3 = df.sample(n=500)
line = alt.Chart(df2).mark_line().encode(
    x=alt.X('actual:Q'),
    y=alt.Y('prediction:Q')
)
scatter = alt.Chart(df3).mark_circle().encode(
    x=alt.X('actual:Q'),
    y=alt.Y('prediction:Q')
)
graph = line + scatter
graph

alt.LayerChart(...)

In [165]:

d = {'actual': [-35, 50], 'prediction': [-35, 50]}
df2 = pd.DataFrame(data=d)


line = alt.Chart(df2).mark_line().encode(
    x=alt.X('actual:Q',
           scale=alt.Scale(domain=(-35, 50))),
    y=alt.Y('prediction:Q',
           scale=alt.Scale(domain=(-35, 50)))
)
scatter = alt.Chart(df3).mark_circle().encode(
    x=alt.X('actual:Q',
           scale=alt.Scale(domain=(-35, 50))),
    y=alt.Y('prediction:Q',
           scale=alt.Scale(domain=(-35, 50)))
)
graph = line + scatter
graph

alt.LayerChart(...)